In [3]:
import numpy as np
import soundfile as sf
import sounddevice as sd
import ipywidgets as wg
from IPython.display import display



#using MIDI note 69 (A440) as a test input
note_test = 69

def MIDI_to_freq(note):
    exp = (note - 69)/ 12
    freq = 2 ** exp * 440
    return freq

def play_audio(data, sr):
    sd.play(data, sr)
    status = sd.wait()
    
def sinusoid(freq, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data


srate = 44100
freq = MIDI_to_freq(69)
data = sinusoid(freq, srate = 44100)


In [11]:
###### testing with Sonja's fucntions to try and bridge the gap between synth and Object tracker ###########
def update_amp(y_prv,y_cur):
    if(y_cur < 0):
        y_cur = 0
    if(y_prv < 0):
        y_prv = 0
    print("previous y: "+ str(y_prv))
    print("current y: "+ str(y_cur))
    
    #minimum movment to change parameter
    if(abs(y_prv-y_cur)>= 5):
        print("changing y position from "+str(y_prv)+" to "+str(y_cur))
        y_cur = y_cur/100
        return y_cur

def update_freq(x_prv,x_cur):
    print("previous x: "+ str(x_prv))
    print("current x: "+ str(x_cur))
    #minimum movment to change parameter
    if(abs(x_prv-x_cur)>= 5):
        print("changing x position from "+str(x_prv)+" to "+str(x_cur))
        return x_cur
    

##this formula decides two things:
## 1: x=0 is 200Hz
## 2: everytime x position changes by 5, frequency changes by 10Hz
## (meaning, minimum freq is 200Hz, max is 2760Hz)
## this function calculates the frequency value, based on the current x position
def new_freq(x_cur):
    i = x_cur//5 #right now freq rounds to nearest int
    freq = 200 + i*10
    return freq

old = 0
new = 20
position = update_freq(old, new)
newfreq = new_freq(position)
print("New frequency is: ", newfreq)
print("Old frequency was: ", new_freq(old))


previous x: 0
current x: 20
changing x position from 0 to 20
New frequency is:  240
Old frequency was:  200


In [18]:
import sys
import ipywidgets as wg
from IPython.display import display

#Sine wav ocsillator that changes frequency in "real-time" based on user text input
#Just type the frequency in Hz you would like to hear then press enter. Enter 0 to quit
 
amp = 1
freq = 440 #frequency
start_idx = 0
block = 500 #the smaller the value, less latency (but if too low input underflow)
lat = 0 #latency in seconds
srate = 2048




try:
    
    def callback(indata, outdata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        global start_idx
         
        t = (start_idx + np.arange(frames)) / srate
        t = t.reshape(-1, 1)
        
        global freq
        global amp
        
        indata = amp * np.sin(2 * np.pi * freq * t) #create sine wave
        outdata[:] = indata  
        start_idx += frames
        
    output1 = wg.Output()
    
            

    with sd.Stream(
                   samplerate=srate, blocksize=block,latency=lat,
                   channels=1, callback=callback ):
        
        print('#' * 80)
        print('press Return to quit')
        print('#' * 80)
        
        def on_value_change(change): #for slider
            with output1:
                print(change.new)
                freq = change.new
                
        f_slider = wg.IntSlider(value = 440, min=0, max = 800)
        display(f_slider, output1)
        
        
        old = int(input("Enter current position: "))
        
        while freq != 0: #get frequency input from user set 0 to quit
            
            
            new = int(input("Enter new position: "))
            position = update_freq(old, new)
            freq = new_freq(position)
            print("New freq is: ", freq)
            old = new
            
            
            
            #freq = int(input("Enter a frequency (0 to quit): "))
            #amp = float(input("Enter amplitude: "))
            
            #f_slider.observe(on_value_change, names = 'value')
            
        input() #return to exit
    
except KeyboardInterrupt:
    
    
    print("Exited by user") 


################################################################################
press Return to quit
################################################################################


IntSlider(value=440, max=800)

Output()

Enter current position: 50
Enter new position: 770
previous x: 50
current x: 770
changing x position from 50 to 770
New freq is:  1740
Enter new position: 640
previous x: 770
current x: 640
changing x position from 770 to 640
New freq is:  1480
Enter new position: 1279
previous x: 640
current x: 1279
changing x position from 640 to 1279
New freq is:  2750
Enter new position: 320
previous x: 1279
current x: 320
changing x position from 1279 to 320
New freq is:  840
Enter new position: 20
previous x: 320
current x: 20
changing x position from 320 to 20
New freq is:  240
Enter new position: 780
previous x: 20
current x: 780
changing x position from 20 to 780
New freq is:  1760
Enter new position: 1279
previous x: 780
current x: 1279
changing x position from 780 to 1279
New freq is:  2750
Enter new position: 


ValueError: invalid literal for int() with base 10: ''

In [14]:
#Taking input from the microphone

import numpy as np
import soundfile as sf
import sounddevice as sd


#using MIDI note 69 (A440) as a test input
note_test = 69

def MIDI_to_freq(note):
    exp = (note - 69)/ 12
    freq = 2 ** exp * 440
    return freq

def play_audio(data, sr):
    sd.play(data, sr)
    status = sd.wait()
    
def sinusoid(freq, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data


srate = 2048


block = 2048
lat = 1
#play_audio(data, srate)


def callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata


#Default device microphone?    
try:
    with sd.Stream(
                   samplerate=srate, blocksize=block,latency=lat,
                   channels=1, callback=callback ):
        print('#' * 80)
        print('press Return to quit')
        print('#' * 80)
        input()
    
except KeyboardInterrupt:
    print("Exited by user")       
        
    

################################################################################
press Return to quit
################################################################################
Exited by user


In [4]:
#I was justy playing around with random widgets in the cells below.

import ipywidgets as wg
from IPython.display import display

name = wg.Text(value='Name')
age = wg.IntSlider(description='Age:')
display(name,age)

Text(value='Name')

IntSlider(value=0, description='Age:')

In [5]:
print(name.value + ' is ' + str(age.value) + ' years old')

Ken is 23 years old


In [6]:
def myPlot(c):
    x = np.linspace(-5,5,20)
    y = c * x**2
    plt.plot(x,y,'r--')
    plt.xlabel('x')
    plt.ylabel('y(x)')
    plt.ylim([0,80])
    plt.xlim([-5,5])
            


In [7]:
c_slide = wg.FloatSlider(value=1.0, min=0, max=3.0, step=0.1)
wg.interact(myPlot,c=c_slide)

interactive(children=(FloatSlider(value=1.0, description='c', max=3.0), Output()), _dom_classes=('widget-inter…

<function __main__.myPlot(c)>

In [91]:
c_slide.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'continuous_update',
 'description',
 'description_tooltip',
 'disabled',
 'layout',
 'max',
 'min',
 'orientation',
 'readout',
 'readout_format',
 'step',
 'style',
 'value']

In [93]:
wg.Widget.widget_types

In [8]:
import time
progress = wg.IntProgress(description='Loading:')
progress.orientation='horizontal'
display(progress)

for i in range(100):
    progress.value = i
    time.sleep(0.1)

IntProgress(value=0, description='Loading:')

In [9]:
int_range = wg.IntSlider()
output2 = wg.Output()

display(int_range, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

In [10]:

caption = wg.Label(value='The values of range1 and range2 are synchronized')
slider = wg.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=1, description='Slider', max=5, min=-5)